# Big Data Analytics - Project - 2021

### Authors: Kirill Holtmann, Lilith Feer, Luca Guenin, Mark Martori Lopez, Remo Hertig
### Matrikel-Nr.: 16722423, 16720799, 16609521, 19759869, 13738323

## Goal Brief
#### The objective of this project is to learn the foundaments of Big Data Analysis and apply them to a big dataset. More precisely with the Cancer Prediction dataset we try to answer the following questions:
#### - Is there a correlation between the genes expressed in the tumor cells? 
#### - Which clinical information is correlated with each other?
#### - Which patients cluster together based on their gene expression?
#### - Is there a difference in gene expression between the clusters?
#### - Is there a difference in gene expression between living or dead patients?

# 

## Packages & Libraries

In [ ]:
install.packages("BiocManager")
BiocManager::install("Rgraphviz")
install.packages("data.table")
install.packages("dplyr")
install.packages("tidyr")
BiocManager::install("outliers")
BiocManager::install("Hmisc")
#install.packages("Hmisc")
install.packages("Seurat")
BiocManager::install("limma")
install.packages("dplyr")
install.packages("corrplot")

In [ ]:
options(warn=0)
library(ggplot2)
library(data.table)
library(dplyr) 
library(tidyr)
library(Hmisc)
library(corrplot)
library(outliers)
library(limma)
library(Seurat)


## Dataset
### Description:
#### Pancreatic cancer arises from the abnormal and uncontrolled growth of cells in the tissues of the pancreas. Pancreatic adenocarcinoma (PAAD) is the most common type of pancreatic cancer, accounting for approximately 85% of all types of pancreatic cancer. This cancer is the twelfth most common cancer and the seventh leading cause of cancer-related death. [01]

#### The dataset is split into 2 subsets. The patients subset consist of 123 rows (features) such as Adenocarcicoma_invasion or days_to_death and 183 columns ( pancreatic cancer tumor cells from patients ). The expression subset contains around 18.500 normalized RNA Sequencing reads for pancreatic cancer tumors as Rows and the same 183 columns as the first subset. The dataset is transposed for easier understanding and manipulation. The file format is GCT , a tab-delimited file used for sharing gene expression data and metadata (details for each sample) for samples.

### Link:
### https://www.kaggle.com/abhiparashar/cancer-prediction?select=PAAD.gct

### Load:

In [ ]:
# - Read in data.table format:                    # Setting Participant ID as column Name -> skip = 3
original_dataset <- fread("Data/PAAD.gct", skip = 3, quote = "", header = TRUE, sep = "\t")

In [ ]:
# Dimensions of the dataset
dimensions <- dim(original_dataset)
message("The dimensions of the dataset are: ","Rows = ",dimensions[1] ," and Columns = ",dimensions[2])

# Overview types of Data
#str(original_dataset)

# Patients IDs
patients_IDs <- colnames(original_dataset)

# Features
rownames <- original_dataset[,1]

# Attributes
#attributes(original_dataset)

head(original_dataset,5)

### Separate the two data sets (meta data and expression data)

In [ ]:
Expression_data <- original_dataset[124:length(original_dataset$participant_id),]
Patient_data <- original_dataset[1:123,]

### Transpose the data sets

In [ ]:
transposefunction <- function(dataset){
    rowname <- dataset$participant_id
    colname <- 1:length(original_dataset[1,])
    transposed_dataset <- transpose(dataset)
    rownames(transposed_dataset) <- colname
    colnames(transposed_dataset) <- rowname
    transposed_dataset <- transposed_dataset[2:nrow(transposed_dataset),]
    return(transposed_dataset)
}

t_expression_data <- transposefunction(Expression_data)
t_patient_data <- transposefunction(Patient_data)

dimensions <- dim(t_expression_data)
message("The dimensions of the Expression dataset are: ","Rows = ",dimensions[1] ," and Columns = ",dimensions[2])
dimensions <- dim(t_patient_data)
message("The dimensions of the Patient dataset are: ","Rows = ",dimensions[1] ," and Columns = ",dimensions[2])

### Missing Values

In [ ]:
# Transform the whole dataset into floats
t_expression_data[] <- lapply(t_expression_data, function(x) {as.numeric(as.character(x))})
dimensionsE <- dim(t_expression_data)

# Expression Data Missing Values Analysis
message("Patients subset:")
tablePatients <- table(is.na(t_patient_data))
dimensionsP <- dim(t_patient_data)
summary(is.na(t_patient_data[1:5,1:5]))

# Expression Data Missing Values Analysis
message("Expression subset:")
tableExpression <- table(is.na(t_expression_data))
summary(is.na(t_expression_data[1:5,1:5]))



message("There are ",tableExpression[2]," missing values from ",dimensionsE[1]*dimensionsE[2]," total values in the expression dataset.")
message("There are ",tablePatients[2]," missing values from ",dimensionsP[1]*dimensionsP[2]," total values in the patients dataset.")

### Visualize the 2 pre-processed subsets:

In [ ]:
head(t_patient_data,5)

In [ ]:
head(t_expression_data[1:5,1:50])

### Gene expression EDA 

To check the gene expression data for obvious data error, we plot the histograms for the minima and maximas for each gene. From these plots we see that the expression values are within biologically reasonable ranges.

In [ ]:
expression_max <- sapply(t_expression_data, max, na.rm=T)
qplot(expression_max, main="Maximum gene expression values") + theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank(),
        panel.background = element_blank(), axis.line = element_line(colour = "black"))

expression_mean <- sapply(t_expression_data, mean, na.rm=T)
qplot(expression_mean, main="Mean gene expression values")+ theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank(),
        panel.background = element_blank(), axis.line = element_line(colour = "black"))

expression_min <- sapply(t_expression_data, min, na.rm=T)
qplot(expression_min, main="Minimum gene expression values") + theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank(),
        panel.background = element_blank(), axis.line = element_line(colour = "black"))

meanE <- mean(expression_min)
sprintf("Mean value found: %f ",meanE)

#### Interpretation of Histograms:
In the hisgorams we see that there are no obvious outliers. The "Maximum gene expression value" is approximately normaly distributed, the "Mean gene expression value" is slightly left skewed and the "Minimum gene expression value" seems to be binomial distributed. To further check for check for outliers we will conduct a Grubbs test.


### Grubbs Test: Check for outliers and Histograms visualization.
If the p-value is less than the chosen significance threshold (generally α = 0.05) then the null hypothesis is rejected and we will conclude that the lowest/highest value is an outlier.


In [ ]:
outlier_fun <- function(df){
  for (columnindex in 1:ncol(df)){
      hist(data.matrix(df[,..columnindex]))
      print(grubbs.test(data.matrix(df[,..columnindex])))
  }
}

#outlier_fun(t_expression_data)

The histograms provided by the Grubbs Test identifies no outliers outside a plausible range, therefore we leave them in the dataset. As there are ~20000 diagrams we decided to not include them in our output file as it would drastically increase the size of our notebook. Yet we have the code to show all the diagrams included as a comment in the above cell. 

## Patient meta data histograms 

### Numeric columns

We investigate and clean up the numeric data part of the patient data

In [ ]:
t_patient_data <- t_patient_data %>%
    mutate_all(type.convert) %>%
    mutate_if(is.factor, as.character)

# only numeric values
df_patient_num <-  t_patient_data %>% select_if(is.numeric)

sprintf("Initially: %d numeric columns",ncol(df_patient_num))

# columns without any variance
which(apply(df_patient_num,2,sd)==0)

# remove dcc columns because they are all the same and refer to the upload of the dataset?

df_patient_num <- df_patient_num %>% select(-day_of_dcc_upload, -month_of_dcc_upload, -year_of_dcc_upload)


# days_to_index,days_to_initial_pathologic_diagnosis, number_of_lymphnodes_positive_by_ihc  is 0 or NA
df_patient_num <- df_patient_num %>% select(-days_to_index, -days_to_initial_pathologic_diagnosis, -number_of_lymphnodes_positive_by_ihc)


#system_version is 6 or 7, for our analysis not relevant
df_patient_num <- df_patient_num %>% select(-system_version)

sprintf("Final: %d numeric columns",ncol(df_patient_num))
#df_patient_num$system_version


In [ ]:
head(df_patient_num)

In [ ]:
ggplot(gather(df_patient_num[,1:9], cols, value), aes(x = value)) + 
       geom_histogram(binwidth = 1) + facet_wrap(.~cols, scales = "free") + theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank(),
        panel.background = element_blank(), axis.line = element_line(colour = "black")) 

In the figures above we can see that most patients have their initial pathalogical diagnosis between 60-70  years of age, which can be approximatly seen as noramly distributed (which makes sense as the sample age goes from ~40 to 90 years of age). The consumption of alcohol per day is clearly right skewed.

In [ ]:
ggplot(gather(df_patient_num[,19:21], cols, value), aes(x = value)) + 
       geom_histogram(binwidth = 1) + facet_wrap(.~cols, scales = "free") + theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank(),
        panel.background = element_blank(), axis.line = element_line(colour = "black"))

In these histograms we can see once again that "year_of_completion" is approximately noramly distributed, Year of initial pathological diagnosis is clearly left skewed (which makes sense, as normaly older people have an initial pathological diagnosis and therefore in our sample it should be in a later year). For the "year_of_tobacco_smoking_onset" we can't see a clear distribution. WE can tell that there are three peaks in the histogram. Those peaks were in ~1960 (which also was the highest peak, which can be explained that smoking was viewed as healthy in those times), ~1969 was another peak but quiet a bit smaller than the first peak. And the last peak was in approximately 1980. 

In [ ]:
ggplot(t_patient_data, 
       aes(y = vital_status, 
           x = age_at_initial_pathologic_diagnosis)) +
  geom_point() + scale_x_discrete(breaks=seq(35,90,10)) + 
  theme(panel.grid.major = element_blank(), 
                       panel.grid.minor = element_blank(),
                      panel.background = element_blank(), 
                      axis.line = element_line(colour = "black"))+ 
  xlab("Age") + ylab("Vital Status") + labs(title = "Vital Status by Age")


In the above depicted figure we see that there is no obvious difference in age between being alive or dead. The above plot provides evidence that in fact there is no obvious difference in being dead or alive by age. More specifically, we do not observe any strong dependency on age, but this question is futher investigated in the following section.



## 
## Correlations - Pearson - Patients Data

In [ ]:
corrPatient <- rcorr(as.matrix(df_patient_num))

corrPatient_plot <- cor(as.matrix(df_patient_num), use="pairwise.complete.obs")

corrPatient_plot[is.na(corrPatient_plot)] <- 0

head(corrPatient,5) # NA Values transformed to 0. For plotting purposes.

In [ ]:
which(apply(df_patient_num,2,sd)==0)

In [ ]:
dpn_na <- df_patient_num %>%  summarise_all(funs(sum(!is.na(.))))
t(dpn_na)

In [ ]:
#complete.cases(dfm)
#dfm

In [ ]:
# This function returns a Table with Rownames, Colnames, Correlation Coefficient, p-value
flattenCorrMatrix <- function(cormat, pmat) {
  ut <- upper.tri(cormat)
  data.frame(
    row = rownames(cormat)[row(cormat)[ut]],
    column = rownames(cormat)[col(cormat)[ut]],
    cor  =(cormat)[ut],
    p = pmat[ut]
    )
}

### Table of correlations

In [ ]:
# Visualize Correlation - Table

dffc <- flattenCorrMatrix(corrPatient$r, corrPatient$P)
dffc

### Plot

In [ ]:
options(jupyter.plot_scale=1, repr.plot.res = 200)
#options(repr.plot.width = 30, repr.plot.height = 30.75, repr.plot.res = 100)

corrplot(corrPatient_plot, type = "lower", order = "hclust", 
         tl.col = "black", tl.srt = 50, tl.cex=0.65)

### Detailed correlation analysis
For a more detailed anaylsis we seperate the dataset into those patient who are still alive and those who are deceased.
Some variables make only sense for these subgroups such as days_to_last_followup and days_to_death which are mutually exclusive

We further remove the variables days_to_pancreatitis_onset and days_to_diabetes_onset because there are too many NaN in these variables


In [ ]:
df_patient_relevant <- t_patient_data %>% select(-day_of_dcc_upload, -month_of_dcc_upload, -year_of_dcc_upload, -days_to_index, -days_to_initial_pathologic_diagnosis, -number_of_lymphnodes_positive_by_ihc,-system_version)


# not enough non NaN in these two variables
df_patient_relevant <- df_patient_relevant %>% select(-days_to_diabetes_onset, -days_to_pancreatitis_onset)

df_patient_dead <-  df_patient_relevant %>% filter(vital_status == 'dead') %>% select_if(is.numeric) %>% select(-days_to_last_followup)
head(df_patient_dead)

df_patient_alive <-  df_patient_relevant %>% filter(vital_status == 'alive') %>% select_if(is.numeric) %>% select(-days_to_death)
head(df_patient_alive)

In [ ]:
cor_dead <- cor.mtest(df_patient_dead, conf.level = 0.95)
M <- cor(df_patient_dead)
corrplot(M, type = "lower", 
         tl.col = "black", tl.srt = 50, tl.cex=0.65, p.mat = cor_dead$p, sig.level = .05)


In [ ]:
cor_alive <- cor.mtest(df_patient_alive, conf.level = 0.95)
M <- cor(df_patient_alive)
corrplot(M, type = "lower", 
         tl.col = "black", tl.srt = 50, tl.cex=0.65, p.mat = cor_alive$p, sig.level = .05)

After controlling for non significant correlation, the only significant relations seems to be negative for age_at_initial_pathologic_diagnosis and days_to_birth. (days_to_birth is a negative number, which makes sense)
There is a positive correlation between days to pancreatitis and diabetes. Pancreatitis is a known complication of diabetes and it therefore makes sense that there's a correlation. 
Number of pack years (years of smoking * packs per day) is correlated negatively with pathologic onset. Therefore, the more cigarettes the patient smokes the earlier the patient is diagnosed with cancer.
Alcohol consumption (amount per day) is negatively correlated to days to death. Therefore, the higher the consumption of alcohol, the earlier the patient dies after diagnosis.

## Correlation - Pearson - Expression Dataset

In [ ]:
# Perform PEARSON correlation
corrExpression <- rcorr(as.matrix(t_expression_data[1:100,1:100]))

# Transform Data for easy plotting
corrExpression[is.na(corrExpression)] <- 0
head(corrExpression,5)


### Table of correlations

In [ ]:
# Table of correlations with p-values
dffcE <- flattenCorrMatrix(corrExpression$r, corrExpression$P)
dffcE[order(dffcE$p, -dffcE$cor),]

### Plot

In [ ]:
options(jupyter.plot_scale=1, repr.plot.res = 200)
corrplot(corrExpression$r, type = "lower", order = "hclust", 
         tl.col = "black", tl.srt = 50, tl.cex=0.65)

A lot of genes, that are highly correlated, are in the same pathway.

### --------------------------------------------------------------------------------------------------------------------------------------------------------------

## Gene expression difference between dead and alive patients

We look if there is any difference in the average gene expressions between dead and alive patients for all genes. Interestingly a gene which is more expressed in the dead group is PSCA (Prostate stem cell antigen) which is known to be associated with cancer in the pancreas. 

In [ ]:
idx_dead <- which( t_patient_data$vital_status %in% 'dead')
dead_gene <- t_expression_data[idx_dead]
idx_alive <- which(t_patient_data$vital_status %in% 'alive')
alive_gene <- t_expression_data[idx_alive,]

alive_means <- colMeans(alive_gene,na.rm=TRUE)
dead_means <- colMeans(dead_gene,na.rm=TRUE)
diff_means <- dead_means - alive_means
sort(diff_means, decreasing = TRUE)[1:10]


Top 10 genes which are on average more expressed in the dead patients

In [ ]:
t(sort(diff_means, decreasing = TRUE)[1:10])

Top 10 genes which are on average less expressed in the dead patients

In [ ]:
t(sort(diff_means, decreasing = FALSE)[1:10])

## Clustering Analysis

In [ ]:
em <- as.matrix(t_expression_data)
tem <- t(em)
colnames(tem) <- paste0("Cell", 1:ncol(tem))
tem

In [ ]:
# Create seurat container and keep all genes that are expressed in at least 3 patients
rna <- CreateSeuratObject(counts = tem, assay = "RNA", min.cells=3)
rna

In [ ]:
# visualize gene and molecule counts
VlnPlot(object = rna, features = c("nFeature_RNA", "nCount_RNA"), ncol = 2)

Visualizing gene and molecule counts is important. In particular, it could be used to define a cut off value. This is nicely depicted in the n_feature_RNA and n_count_RNA since one could implement a cut_off value around 14'080 in the former and around 121'500 in the latter. 

In [ ]:
# data is already normalized
#pbmc <- NormalizeData(object = rna, normalization.method = "LogNormalize", scale.factor = 10000)
#pbmc

Normalizing the data is often necessary for removing technological factors, such as human imprecision, that could create differences. Indeed, we would like to keep only the biological differences. 

We detect variable genes (calculates the average expression and dispersion for each gene, places these genes into bins, and then calculates a z-score for dispersion within each bin) and we control for the relationship between variability and average expression. We focus on these genes during downstream analysis.

In [ ]:
pbmc <- FindVariableFeatures(object = rna, mean.function = ExpMean, dispersion.function = LogVMR, x.low.cutoff = 0.0125, x.high.cutoff = 3, y.cutoff = 0.5, nfeatures = 2000)
# see the most variable genes
head(x = HVFInfo(object = pbmc))

In [ ]:
#?FindVariableFeatures

There are no mitochondrial genes present in the data set. They were removed during the study from which the dataset stems from.
Mitochondrial genes are often used to detect damaged cells, which should be excluded during analysis. 

In [ ]:
mito.genes <- grep(pattern = "^MT-", x = rownames(pbmc@assays[["RNA"]]), value = TRUE)
mito.genes

In [ ]:
 #grep(pattern = "^MT-", x = rownames(tem), value = TRUE)

We regress out cell-cell variation in gene expression driven by batch, cell alignment rate and 
the number of detected molecules

In [ ]:
pbmc <- ScaleData(object = pbmc, vars.to.regress = c("nCount_RNA", "nFeature_RNA"))

## Principal Component Analysis

In [ ]:
pbmc <- RunPCA(object = pbmc,  npcs = 50, verbose = FALSE)
DimPlot(object = pbmc, reduction = "pca")

There is only one accumulation of data points present, but they still may be sorted by variables from the meta data (live/dead, position of the tumor, staging of the tumor etc.)

## Graph based clustering

In [ ]:
pbmc <- FindNeighbors(pbmc, reduction = "pca", dims = 1:50)
pbmc <- FindClusters(pbmc, resolution = 0.65, algorithm = 1)

##### Using the resolution 0.65, two clusters were found. Hence, we can compare gene expression between them

In [ ]:
pbmc <- RunUMAP(pbmc, reduction = "pca", dims = 1:20)

### Clusters visualization

In [ ]:
#DimPlot(pbmc, reduction = "umap", split.by = "seurat_clusters")
DimPlot(pbmc, reduction = "umap")

In this UMAP we see that there have been 2 clusters generated. Unfortunately these two clusters seem to not show any pattern at all. This might be due to data prepartion by the creator of the dataset.

## Differential Expression between the clusters
Differential expression identifies positive and negative markers of a single cluster

### Markers of cluster 0

In [ ]:
# find all markers of cluster 0
cluster1.markers <- FindMarkers(object = pbmc, ident.1 = 0, min.pct = 0.25, logfc.threshold=0.25)
print(x = head(x = cluster1.markers, n = 5))

In [ ]:
#cluster1.markers[order(-cluster1.markers$avg_logFC),]

The genes "TJP3", "GRTP1", "MICALL2", "TRPM4" and "C2orf29" show a higher expression in cluster 0 which can also be seen in the violin plots below.

In [ ]:
VlnPlot(object = pbmc, features =c("TJP3", "GRTP1", "MICALL2", "TRPM4", "C2orf29"))

TJP3 is a protein in tight junctions, which are often found in epithelial cells. \
GRTP1 is an GTPase-activating protein for Rab family proteins. \
MICALL2 is important for actin cytoskeleton reorganization, which can enable cancer metastasis. \
TRPM4 is a calcium-activated non selective (CAN) cation channel that mediates membrane depolarization, which is known to promote cell proliferation. \
C2orf29 is important for mRNA decay and therefore gene expression regulation. Patients in this cluster are likely to have a larger protein decay. \
\
Furthermore we see in the violin plots of marker 0, that most of the time the two identities of the genes are rather similar in size and variation. We can also observe that the identity 1 is likely to deviate downards when compared to identity 1 in all of the violin plots. Only the violin plot of gene "MICALL2" seems to be different from the rest (although also being rather similar to identity 1 in variation but not so much in size. Also in this violin polt there seems to be an upward deviation.

### Markers of cluster 1

In [ ]:
# find all markers of cluster 1
cluster2.markers <- FindMarkers(object = pbmc, ident.1 = 1, min.pct = 0.25, logfc.threshold=0.25)
print(x = head(x = cluster2.markers, n = 5))

In [ ]:
subset <- cluster2.markers[ which(cluster2.markers$p_val_adj < 0.05),]
subset <- subset[order( -subset$avg_log2FC),]
head(subset, n = 5)

The genes "ADCY1", "ATP8A2", "P2RX5", "PEG3" and "GPX3" show a higher expression in cluster 1 which can also be seen in the violin plots below.

In [ ]:
VlnPlot(object = pbmc, features =c("ADCY1", "ATP8A2", "P2RX5", "PEG3", "GPX3"))

ADCY1 is a gene that is often found in chemotherapy resistance. \
Glutathionperoxidase (GPX3) is important as antioxidant. It has an ambivalent role in cancer since it is a tumor suppressor and pro-survival protein. \
In this set of violin plots of cluster 1 we see basically the same as in cluster 0 with the difference that identity 1 seems to deviated upwards instead of downwards (as seen in cluster 0). Furthermore we see again that the two identities of genes are rather similar in size and variation.

## Color by metadata

### histological_type

In [ ]:
hist_clusters <- as.factor(t_patient_data$histological_type)
pbmc <- AddMetaData(pbmc, hist_clusters, col.name="hist_clusters")

In [ ]:
DimPlot(pbmc, reduction = "umap", group.by = "hist_clusters")

In this UMAP we see that pancrease-adenocarcinoma ductal type is most present. This means that most of the patients have this type of pancreas-adenocarcinoma. Unfortunately we cannot see any other patterns in the UMAP and can therefore make no further interpretations.

### pathologic_stage

In [ ]:
pathologic_stage_clusters <- as.factor(t_patient_data$pathologic_stage)
pbmc <- AddMetaData(pbmc, pathologic_stage_clusters, col.name="pathologic_stage")
DimPlot(pbmc, reduction = "umap", group.by = "pathologic_stage")

In this pathologic state UMAP we can again only see one pattern which is that the "stage iib" is most present. Other than that we can once again not recognize any other patterns in the UMAP.

### tissue_source_site

In [ ]:
tissue_source_site_clusters <- as.factor(t_patient_data$tissue_source_site)
pbmc <- AddMetaData(pbmc, tissue_source_site_clusters, col.name="tissue_source_site")
DimPlot(pbmc, reduction = "umap", group.by = "tissue_source_site")

In the tissue_source_site UMAP we cannot recognize any patterns at all. The distribution of all tissue_source_site seems to be completely random.

### Vital Status 

We now would like to figure out if there are pattern recognizable in the UMAPS if it is colored by vital status. Yet, the simply binary variable vital_status does not take into account the lenght since which a person alive has been suffering from cancer. In other words, her/his cancer could have just been diagnozed. Hence, we differenriated between indivuals that passed away, indiviudals that are alive with their last follow up check up having occured at least 400 days ago (alive) and individuals alive but with a last follow up check up having occured less than 400 days ago (new alive). We tested for different threhold, with similar results in each specification.

In [ ]:
t_patient_data <- t_patient_data %>%
  mutate(status_updated = case_when(
    t_patient_data$vital_status == "dead"~"dead",
    t_patient_data$vital_status == "alive" & t_patient_data$days_to_last_followup <= 400 ~ "alive new", 
    t_patient_data$vital_status == "alive" & t_patient_data$days_to_last_followup >= 400~ "alive"))

status <- as.factor(t_patient_data$status_updated)
pbmc <- AddMetaData(pbmc, status, col.name="status_updated")
DimPlot(pbmc, reduction = "umap", group.by = "status_updated")

Overall, there is no pattern recognizable in the UMAPs colored by tissue_source_site, pathologic_stage, hist_clusters and vital_status. This may be due to the fact that we only have average expression per patient and any cell specific differences are averaged out. Often in single cell UMAP plots, bigger differences can be observed.

## Conclusions

### Gene Expression Correlations
We did find some correlated genes in the expression data. We suspect they are correlated because they are part of the same pathway, e.g ABCA8, ABCA6 & ABCA9 are part of the Glucose transport pathway [2]


### Patient Clustering based on Gene Expression
The patients gene expression data was clustered using the Seurat R package. This package uses a graph-based clustering method where the gene expressions are reduced with PCA and then converted into a graph based on a K-nearest neighbour method. Then a community detection algorithm tries to partition the graph into clusters.

We suspect that this method is appropriate for analyzing many cell samples. However in our dataset it appears that the gene expression of the cell samples from the patients have been averaged. This limits the validity and interpretability our our differential gene expression analysis to some extent.

Nontheless, cluster analysis revelaled some genes which seem to be associated with cancer such as ADCY1, GPX3 and MICALL2.


### Gene Expression Difference between dead and alive patients
A large difference in the over-expression of the PSCA gene in dead patients was found. The PSCA gene is known to be associated with pancreatic cancer [3].



## References

#### 01 - Baek, B., Lee, H. Prediction of survival and recurrence in patients with pancreatic cancer by integrating multi-omics data. Sci Rep 10, 18951 (2020). https://doi.org/10.1038/s41598-020-76025-1

#### 2 Transport of glucose and other sugars, bile salts and organic acids, metal ions and amine compounds https://pathcards.genecards.org/card/transport_of_glucose_and_other_sugars_bile_salts_and_organic_acids_metal_ions_and_amine_compounds

#### 3 Reiter RE, Gu Z, Watabe T, et al. Prostate stem cell antigen: a cell surface marker overexpressed in prostate cancer. Proc Natl Acad Sci U S A. 1998;95(4):1735-1740. doi:10.1073/pnas.95.4.1735